In [1]:
import os
import time
import numpy as np
import cv2
from glob import glob
from matplotlib import pyplot
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, BatchNormalization,Embedding,Dense
from tensorflow.keras.layers import Reshape,Concatenate


import warnings
warnings.filterwarnings('ignore')


2024-04-11 18:47:53.124492: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 18:47:53.124630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 18:47:53.256536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
IMG_H=128
IMG_W=128
IMG_C=3
STRIDE = 32


In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_png(img)
    img = tf.image.resize(img, [IMG_H, IMG_W])
    img = tf.cast(img, tf.float32)
    img = (img - 127.5) / 127.5
    return img, label

def tf_dataset(images_path, images_label, batch_size):
    ds = tf.data.Dataset.from_tensor_slices((images_path, images_label))
    ds = ds.shuffle(buffer_size=1000).map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds


In [4]:
def conv_block(x, num_filters, kernel_size=4, padding="same", strides=1, activation=True):
    x = L.Conv2D(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02),
        padding=padding,
        strides=strides,
        use_bias=False
    )(x)

    if activation:
        x = L.LeakyReLU(alpha=0.2)(x)
        x = L.Dropout(0.3)(x)
    return x

def deconv_block(x, num_filters, kernel_size=5, padding="same", strides=4):
    x = L.Conv2DTranspose(
        num_filters,
        kernel_size=kernel_size,
        strides=strides,
        padding=padding,
        kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02),
        use_bias=False
    )(x)
    x = L.BatchNormalization()(x)
    x = L.LeakyReLU(alpha=0.2)(x)
    return x

In [5]:
def build_generator(latent_dim, embed_dim, num_classes):
    noise = L.Input((latent_dim,), name="noise_input")
    x = L.Dense(IMG_H//STRIDE * IMG_W//STRIDE * 256)(noise)
    x = L.BatchNormalization()(x)
    x = L.LeakyReLU(0.2)(x)
    x = L.Reshape((IMG_H//STRIDE, IMG_W//STRIDE, 256))(x)

    label = L.Input((1,), name="class_label")  # Ensure shape is passed as a tuple
    l = L.Embedding(num_classes, embed_dim)(label)
    l = L.Dense(IMG_H//STRIDE * IMG_W//STRIDE * 256)(l)
    l = L.BatchNormalization()(l)
    l = L.LeakyReLU(0.2)(l)
    l = L.Reshape((IMG_H//STRIDE, IMG_W//STRIDE, 256))(l)

    # Concatenate noise and label embeddings
    x = L.Concatenate()([x, l])

    x = deconv_block(x, 512, kernel_size=5, strides=2)
    x = deconv_block(x, 256, kernel_size=5, strides=2)
    x = deconv_block(x, 128, kernel_size=5, strides=2)
    x = deconv_block(x, 64, kernel_size=5, strides=2)
    x = deconv_block(x, 32, kernel_size=5, strides=2)

    x = conv_block(
        x,
        num_filters=3,
        kernel_size=5,
        strides=1,
        activation=False
    )

    fake_output = L.Activation("tanh")(x)

    return Model([noise, label], fake_output, name="generator")


In [6]:
def build_discriminator(embed_dim, num_classes):
    image = L.Input((IMG_H, IMG_W, IMG_C), name="disc_input")

    label = L.Input((1,), name="class_label")
    l = L.Embedding(num_classes, embed_dim)(label)
    l = L.Dense(IMG_H * IMG_W * 1)(l)
    l = L.BatchNormalization()(l)
    l = L.LeakyReLU(0.2)(l)
    l = L.Reshape((IMG_H, IMG_W, 1))(l)

    x = L.Concatenate()([image, l])

    x = conv_block(x, 32, kernel_size=5, strides=2)
    x = conv_block(x, 64, kernel_size=5, strides=2)
    x = conv_block(x, 128, kernel_size=5, strides=2)
    x = conv_block(x, 256, kernel_size=5, strides=2)
    x = conv_block(x, 512, kernel_size=5, strides=2)

    x = L.Flatten()(x)
    x = L.Dense(1)(x)

    return Model([image, label], x, name="discriminator")

In [7]:
def train_step(real_images, real_labels, latent_dim, num_classes, generator, discriminator, g_opt, d_opt):
    batch_size = tf.shape(real_images)[0]
    bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)

    ## Discriminator
    noise = tf.random.normal([batch_size, latent_dim])

    for _ in range(3):
        with tf.GradientTape() as dtape:
            generated_images = generator([noise, real_labels], training=True)

            real_output = discriminator([real_images, real_labels], training=True)
            fake_output = discriminator([generated_images, real_labels], training=True)

            d_real_loss = bce_loss(tf.ones_like(real_output), real_output)
            d_fake_loss = bce_loss(tf.zeros_like(fake_output), fake_output)
            d_loss = d_real_loss + d_fake_loss

            d_grad = dtape.gradient(d_loss, discriminator.trainable_variables)
            d_opt.apply_gradients(zip(d_grad, discriminator.trainable_variables))

    with tf.GradientTape() as gtape:
        generated_images = generator([noise, real_labels], training=True)

        fake_output = discriminator([generated_images, real_labels], training=True)

        g_loss = bce_loss(tf.ones_like(fake_output), fake_output)

        g_grad = gtape.gradient(g_loss, generator.trainable_variables)
        g_opt.apply_gradients(zip(g_grad, generator.trainable_variables))

    return d_loss, g_loss

In [8]:
def save_plot(examples, epoch, n):
    n = int(n)
    examples = (examples + 1) / 2.0
    examples = examples * 255
    file_name = f"D:\MINI PROJECT\GENERATED OUTPUT\generated_plot_epoch-{epoch+1}.png"

    cat_image = None
    for i in range(n):
        start_idx = i*n
        end_idx = (i+1)*n

        image_list = examples[start_idx:end_idx]
        if i == 0:
            cat_image = np.concatenate(image_list, axis=1)
        else:
            tmp = np.concatenate(image_list, axis=1)
            cat_image = np.concatenate([cat_image, tmp], axis=0)

    cat_image = cv2.cvtColor(cat_image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(file_name, cat_image)

In [9]:
if __name__ == "__main__":
    """ Hyperparameters """
    batch_size = 32
    latent_dim = 100
    embed_dim = 50
    num_classes = 2
    num_epochs = 1000
    n_samples = 25
    path = r"D:\MINI PROJECT\Five Classes"

    """ Images """
    images_path = glob(f"{path}/*/*.png")

    """ Labels """
    labels_list = os.listdir(f"{path}")
    print(f"Number of labels: {len(labels_list)}")
    print(f"Labels: {labels_list}")

    images_label = []
    for path in images_path:
        name = path.split("/")[-2]
        index = labels_list.index(name)
        images_label.append(index)

    print(f"Images: {len(images_path)} - Labels: {len(images_label)}")

    """ Folders """
    create_dir("samples")
    create_dir("new_model")

    """ Model """
    g_model = build_generator(latent_dim, embed_dim, num_classes)
    d_model = build_discriminator(embed_dim, num_classes)

    # g_model.load_weights("saved_model/g_model.h5")
    # d_model.load_weights("saved_model/d_model.h5")


    """ Training """
    d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

    images_dataset = tf_dataset(images_path, images_label, batch_size)
    seed = np.random.normal(size=(n_samples, latent_dim))

    seed_class_label = [0]*1+ [1]*1+ [2]*1 + [3]*1 + [4]*1
    seed_label = []
    for item in seed_class_label:
        seed_label += [item] * int(np.sqrt(n_samples))

    seed_label = np.array(seed_label)

    for epoch in range(num_epochs):
        start = time.time()

        d_loss = 0.0
        g_loss = 0.0
        for image_batch, label_batch in images_dataset:
            d_batch_loss, g_batch_loss = train_step(image_batch, label_batch, latent_dim, num_classes, g_model, d_model, g_optimizer, d_optimizer)
            d_loss += d_batch_loss
            g_loss += g_batch_loss

        d_loss = d_loss/len(images_dataset)
        g_loss = g_loss/len(images_dataset)

        g_model.save("new_model/g_model.keras")
        d_model.save("new_model/d_model.keras")

        examples = g_model.predict([seed, seed_label], verbose=0)
        save_plot(examples, epoch, np.sqrt(n_samples))

        time_taken = time.time() - start
        print(f"[{epoch+1:1.0f}/{num_epochs}] {time_taken:2.2f}s - d_loss: {d_loss:1.4f} - g_loss: {g_loss:1.4f}")


    ##

Number of labels: 5
Labels: ['maizee', 'shepherd', 'fat-hen', 'small-flowered-cranesbill', 'charlock']
Images: 1035 - Labels: 1035


I0000 00:00:1712861335.748380      69 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[1/1000] 49.74s - d_loss: 0.8771 - g_loss: 1.4163
[2/1000] 37.39s - d_loss: 1.0915 - g_loss: 1.0995
[3/1000] 36.74s - d_loss: 1.1089 - g_loss: 1.3907
[4/1000] 36.71s - d_loss: 1.0367 - g_loss: 1.1541
[5/1000] 36.41s - d_loss: 1.0393 - g_loss: 1.1461
[6/1000] 36.58s - d_loss: 0.9745 - g_loss: 1.3349
[7/1000] 36.69s - d_loss: 0.8857 - g_loss: 1.5254
[8/1000] 36.63s - d_loss: 0.8101 - g_loss: 1.6128
[9/1000] 36.70s - d_loss: 0.7522 - g_loss: 1.7511
[10/1000] 36.56s - d_loss: 0.7619 - g_loss: 1.7356
[11/1000] 36.44s - d_loss: 0.7161 - g_loss: 1.9283
[12/1000] 36.62s - d_loss: 0.6899 - g_loss: 2.0053
[13/1000] 36.77s - d_loss: 0.7539 - g_loss: 1.9017
[14/1000] 36.71s - d_loss: 0.6839 - g_loss: 2.0573
[15/1000] 36.86s - d_loss: 0.6895 - g_loss: 1.9960
[16/1000] 37.08s - d_loss: 0.6858 - g_loss: 1.8935
[17/1000] 37.11s - d_loss: 0.7219 - g_loss: 1.9119
[18/1000] 36.96s - d_loss: 0.6472 - g_loss: 2.0829
[19/1000] 37.11s - d_loss: 0.7264 - g_loss: 1.8689
[20/1000] 36.95s - d_loss: 0.7421 - g_lo